<br>
<u>Notebook Two</u> | 
<a href=https://leone.gdn/DLL target=_blank>Report</a> | 
<a href=https://github.com/andrealeone/DLL>Repository</a>
<br><br>
<b>Task 1: Attribute classifiers</b><br><br>
Andrea Leone, Alessandro E. Piotti<br>
Deep Learning Lab — University of Trento<br>
December 2021
<hr><br><br>

In [1]:
import project 

import os
import pickle
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import random
import collections

import torch
import torch.nn          as nn
import torch.optim       as optim
import torchvision
import sklearn

from tqdm.notebook   import tqdm

project.notebook()

In [2]:
images, annotations = project.load_resources() 

<hr><br><br>

## `Age` classifiers

<br><br>Load the train set, isolate the attribute as label, and compute class weights

In [3]:
attribute  = 'age' 

split_val  = 640                # 751-640=151
dictionary = dict()
irm        = torchvision.io.ImageReadMode.RGB

for image_file in tqdm(images):
    
    image_id = int(image_file.split('/')[-1].split('_')[0])
    
    label    = annotations.loc[image_id][attribute] - 1
    tensor   = torchvision.io.read_image( image_file, irm ).float()

    channels, height, width = tensor.shape
    tensor    = tensor.reshape(1, channels, height, width)
    
    if image_id not in dictionary:
        dictionary[image_id] = list()
        dictionary[image_id].append([tensor, label])
    else:
        dictionary[image_id].append([tensor, label])
    
data = list(dictionary.items())
random.Random(3).shuffle(data)

train_set, test_set = project.split_dataset(data, split_val)

project.inspect_dataset(data, train_set, test_set) 

  0%|          | 0/12989 [00:00<?, ?it/s]

data:     Counter({1: 10660, 2: 2039, 3: 186, 0: 104})
train:    Counter({1: 9159, 2: 1798, 3: 125, 0: 88})
test:     Counter({1: 1501, 2: 241, 3: 61, 0: 16})


<br>

As age is composed of four classes, we decompose this classifier into four binary ones.<br>This way, in training we define `f` to refer to a particular class and re-organise labels into in-group and out-group with the `project.split(set)` function.

<hr><br><br>Fit a pretrained ResNet50 architecture

In [ ]:
v = '3_2'                       # model version 
e =  0                          # starting epoch
f =  0                          # feature index


model     = torchvision.models.resnet50(pretrained=True) 
model.fc  = nn.Linear(
    in_features=model.fc.in_features,
    out_features=2, bias=True
)

if e > 0:
    w = './models/{}_resnet50.v{}.x{}.net'.format(attribute, v, e)
    model.load_state_dict( torch.load(w) )


optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for i in range(10): 

    model_name = '{}_resnet50.v{}.x{}.net'.format(attribute, v, e+i+1)

    p = project.train(model,
        project.split(train_set, label=f),
        criterion, optimizer, epochs=1, li=500, monobatch=True
    )

    r = project.test(model,
        project.split(test_set, label=f), monobatch=True
    )

    project.save(model, './models/{}'.format(model_name))

In [ ]:
# SCORE BOARD 
# 
# net_design_name    | epochs | accuracy           | precision | recall
#                    |        |                    |           |
# resnet50 v3.2 f0   |  1     | 0.40625            | 0.3961039 | 0.40625
# resnet50 v3.2 f0   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  3     | 0.46875            | 0.2419355 | 0.46875
# resnet50 v3.2 f0   |  4     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  5     | 0.34375            | 0.2714286 | 0.34375
# resnet50 v3.2 f0   |  6     | 0.5625             | 0.7666667 | 0.5625
# resnet50 v3.2 f0   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   |  8     | 0.6875             | 0.6904762 | 0.6875
# resnet50 v3.2 f0   |  9     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 10     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 11     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 12     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 13     | 0.59375            | 0.7758621 | 0.59375
# resnet50 v3.2 f0   | 14     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f0   | 15     | 0.5                | 0.25      | 0.5
#                    |        |                    |           |
# resnet50 v3.2 f1   |  1     | 0.8251786695986806 | 0.4125893 | 0.5
# resnet50 v3.2 f1   |  2     | 0.8251786695986806 | 0.4125893 | 0.5
# resnet50 v3.2 f1   |  3     | 0.8251786695986806 | 0.4125893 | 0.5
# resnet50 v3.2 f1   |  4     | 0.8251786695986806 | 0.4125893 | 0.5
# resnet50 v3.2 f1   |  5     | 0.8246289169873557 | 0.5793869 | 0.5009061
# resnet50 v3.2 f1   |  6     | 0.8125343595382079 | 0.6091305 | 0.5381894
# resnet50 v3.2 f1   |  7     | 0.8185816382627817 | 0.6164298 | 0.5257438
# resnet50 v3.2 f1   |  8     | 0.8273776800439802 | 0.6992081 | 0.5223991
# resnet50 v3.2 f1   |  9     | 0.8251786695986806 | 0.6757279 | 0.5842667
#                    |        |                    |           |
# resnet50 v3.2 f2   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f2   |  2     | 0.5767634854771784 | 0.6400415 | 0.5767635
# resnet50 v3.2 f2   |  3     | 0.5020746887966805 | 0.7505198 | 0.5020747
# resnet50 v3.2 f2   |  4     | 0.5165975103734440 | 0.5669444 | 0.5165975
# resnet50 v3.2 f2   |  5     | 0.5477178423236515 | 0.5480488 | 0.5477178
# resnet50 v3.2 f2   |  6     | 0.5726141078838174 | 0.6037337 | 0.5726141
# resnet50 v3.2 f2   |  7     | 0.6307053941908713 | 0.6312840 | 0.6307054
# resnet50 v3.2 f2   |  8     | 0.6307053941908713 | 0.6322446 | 0.6307054
# resnet50 v3.2 f2   |  9     | 0.6078838174273858 | 0.6201488 | 0.6078838
# resnet50 v3.2 f2   | 10     | 0.5995850622406639 | 0.6302937 | 0.5995851
# resnet50 v3.2 f2   | 11     | 0.6514522821576764 | 0.6525026 | 0.6514523
# resnet50 v3.2 f2   | 12     | 0.6161825726141079 | 0.6397680 | 0.6161826
#                    |        |                    |           |
# resnet50 v3.2 f3   |  1     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  2     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  3     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  4     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  5     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  6     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  7     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  8     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   |  9     | 0.4754098360655737 | 0.2436975 | 0.4754098
# resnet50 v3.2 f3   | 10     | 0.4754098360655737 | 0.4753834 | 0.4754098
# resnet50 v3.2 f3   | 11     | 0.4836065573770492 | 0.4836022 | 0.4836066
# resnet50 v3.2 f3   | 12     | 0.4344262295081967 | 0.4342672 | 0.4344262
# resnet50 v3.2 f3   | 13     | 0.5                | 0.25      | 0.5
# resnet50 v3.2 f3   | 14     | 0.5081967213114754 | 0.7520661 | 0.5081967
# resnet50 v3.2 f3   | 15     | 0.5573770491803278 | 0.6748567 | 0.5573770
# resnet50 v3.2 f3   | 16     | 0.5245901639344263 | 0.6136646 | 0.5245902
# resnet50 v3.2 f3   | 17     | 0.4672131147540984 | 0.4664835 | 0.4672131
# resnet50 v3.2 f3   | 18     | 0.5409836065573771 | 0.7606838 | 0.5409836
# resnet50 v3.2 f3   | 19     | 0.5081967213114754 | 0.5854342 | 0.5081967
# resnet50 v3.2 f3   | 20     | 0.4918032786885246 | 0.4917945 | 0.4918033
#

<hr><br><br>Fit the custom CNN architecture

In [ ]:
v = '4'                         # model version 
e =  0                          # starting epoch
f =  0                          # feature index


model = project.CNN(6144)

if e > 0:
    w = './models/{}_3cnv_5fc.v{}.x{}.net'.format(attribute, v, e)
    model.load_state_dict( torch.load(w) )


optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for i in range(10): 

    model_name = '{}_3cnv_5fc_f{}.v{}.x{}.net'.format(attribute, f, v, e+i+1)

    p = project.train(model,
        project.split(train_set, label=f),
        criterion, optimizer, epochs=1, li=500, monobatch=True
    )

    r = project.test(model,
        project.split(test_set, label=f), monobatch=True
    )

    project.save(model, './models/{}'.format(model_name))

In [ ]:
# SCORE BOARD 
# 
# net_design_name    | epochs | accuracy           | precision | recall
#                    |        |                    |           |
# c.cnn 3cnv_5fc f0  |  1     | 0.90625            | 0.9210526 | 0.90625
# c.cnn 3cnv_5fc f0  |  2     | 0.46875            | 0.4653680 | 0.46875
# c.cnn 3cnv_5fc f0  |  3     | 0.65625            | 0.7285714 | 0.65625
# c.cnn 3cnv_5fc f0  |  4     | 0.5                | 0.5       | 0.5
# c.cnn 3cnv_5fc f0  |  5     | 0.34375            | 0.2037037 | 0.34375
# c.cnn 3cnv_5fc f0  |  6     | 0.53125            | 0.5386473 | 0.53125
# c.cnn 3cnv_5fc f0  |  7     | 0.40625            | 0.4028340 | 0.40625
# c.cnn 3cnv_5fc f0  |  8     | 0.625              | 0.7051282 | 0.625
# c.cnn 3cnv_5fc f0  |  9     | 0.59375            | 0.6159420 | 0.59375
# c.cnn 3cnv_5fc f0  | 10     | 0.53125            | 0.5457143 | 0.53125
#                    |        |                    |           |
# c.cnn 3cnv_5fc f1  |  1     | 0.8251786695986806 | 0.4125893 | 0.5
# c.cnn 3cnv_5fc f1  |  2     | 0.8251786695986806 | 0.4125893 | 0.5
# c.cnn 3cnv_5fc f1  |  3     | 0.8229796591533810 | 0.6641020 | 0.5668244
# c.cnn 3cnv_5fc f1  |  4     | 0.8141836173721825 | 0.6480121 | 0.5887574
# c.cnn 3cnv_5fc f1  |  5     | 0.8273776800439802 | 0.7358330 | 0.5124854
# c.cnn 3cnv_5fc f1  |  6     | 0.7844969763606378 | 0.6358767 | 0.6463615
# c.cnn 3cnv_5fc f1  |  7     | 0.8279274326553051 | 0.6869648 | 0.5512342
# c.cnn 3cnv_5fc f1  |  8     | 0.7234744365035733 | 0.5989448 | 0.6378882
# c.cnn 3cnv_5fc f1  |  9     | 0.8103353490929082 | 0.5965808 | 0.5331393
# c.cnn 3cnv_5fc f1  | 10     | 0.8042880703683343 | 0.5977102 | 0.5455849
#                    |        |                    |           |
# c.cnn 3cnv_5fc f2  |  1     | 0.6286307053941909 | 0.6337164 | 0.6286307
# c.cnn 3cnv_5fc f2  |  2     | 0.5912863070539419 | 0.6089333 | 0.5912863
# c.cnn 3cnv_5fc f2  |  3     | 0.6804979253112033 | 0.6809465 | 0.6804979
# c.cnn 3cnv_5fc f2  |  4     | 0.6804979253112033 | 0.6815105 | 0.6804979
# c.cnn 3cnv_5fc f2  |  5     | 0.6929460580912863 | 0.6929593 | 0.6929461
# c.cnn 3cnv_5fc f2  |  6     | 0.6369294605809128 | 0.6418760 | 0.6369295
# c.cnn 3cnv_5fc f2  |  7     | 0.6161825726141079 | 0.6207761 | 0.6161826
# c.cnn 3cnv_5fc f2  |  8     | 0.6721991701244814 | 0.6765895 | 0.6721992
# c.cnn 3cnv_5fc f2  |  9     | 0.6597510373443983 | 0.6597510 | 0.6597510
# c.cnn 3cnv_5fc f2  | 10     | 0.6804979253112033 | 0.6809465 | 0.6804979
#                    |        |                    |           |
# c.cnn 3cnv_5fc f3  |  1     | 0.5901639344262295 | 0.6307992 | 0.5901639
# c.cnn 3cnv_5fc f3  |  2     | 0.6721311475409836 | 0.7374861 | 0.6721311
# c.cnn 3cnv_5fc f3  |  3     | 0.6065573770491803 | 0.6108471 | 0.6065574
# c.cnn 3cnv_5fc f3  |  4     | 0.5901639344262295 | 0.6243975 | 0.5901639
# c.cnn 3cnv_5fc f3  |  5     | 0.5737704918032787 | 0.5848007 | 0.5737705
# c.cnn 3cnv_5fc f3  |  6     | 0.6639344262295082 | 0.6650433 | 0.6639344
# c.cnn 3cnv_5fc f3  |  7     | 0.6967213114754098 | 0.7231707 | 0.6967213
# c.cnn 3cnv_5fc f3  |  8     | 0.5655737704918032 | 0.5884058 | 0.5655738
# c.cnn 3cnv_5fc f3  |  9     | 0.6065573770491803 | 0.6869401 | 0.6065574
# c.cnn 3cnv_5fc f3  | 10     | 0.5245901639344263 | 0.5290938 | 0.5245902
#

<br><br>